In [22]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd
import re
from datetime import datetime

In [2]:
def read_gene_sequences(file_gene_sequences):
    multiple_genes = []
    with open(file_gene_sequences) as handle:
        for values in SeqIO.FastaIO.SimpleFastaParser(handle):
            sequence_id, sequence = values
            multiple_genes.append((sequence_id, sequence))

    return multiple_genes

In [34]:
def chave_ordenacao(item):
    # Extrai a data da string e converte para um objeto datetime
    data_str = item.id.split('_')[-1]
    data = datetime.strptime(data_str, "%Y-%m-%d")
    return data

In [37]:
# Diretório principal onde você possui várias pastas
main_directory = '/home/m_souza/tcc/dataset/new_dataset_vagner'

# Iterar sobre as pastas no diretório principal
for pasta in os.listdir(main_directory):
    pasta_path = os.path.join(main_directory, pasta)

    # Verificar se é um diretório
    if os.path.isdir(pasta_path):
        # Procurar pelo arquivo sequencias_cortadas.fasta na pasta
        arquivo_sequencias = os.path.join(pasta_path, 'sequencias_spike_final.fasta')
        arquivo_csv = os.path.join(pasta_path, 'BVBRC_genome.csv')

        path_sequencias_novas = os.path.join(pasta_path, 'sequencias_spike_final_ok.fasta')

        # Verificar se o arquivo existe
        if os.path.exists(arquivo_sequencias):
            print(f'Analisando sequências em {pasta}')

            # Ler as sequências do arquivo usando a Biopython
            sequencias = read_gene_sequences(arquivo_sequencias)

            # Lê o arquivo CSV
            dados_planilha = pd.read_csv(arquivo_csv)

            sequencias_novas = []
            for seq_tuple in sequencias:
                seq_id, seq_sequence = seq_tuple
                accn = seq_id.split('_')[1]

                # Filtra os dados onde a coluna 'GenBank Accessions' é igual ao valor de accn
                linha_selecionada = dados_planilha[dados_planilha['GenBank Accessions'] == accn]
                if not linha_selecionada.empty:
                    # Obtém os valores das colunas 'Isolation Country' e 'Collection Date'
                    collection_date = linha_selecionada['Collection Date'].values[0]
                    # Aplica o filtro de data usando expressão regular
                    if pd.notna(collection_date) and re.match(r'\d{4}-\d{2}-\d{2}', collection_date):
                        isolation_country = linha_selecionada['Isolation Country'].values[0]

                        # Concatena os valores com accn
                        novo_id = f"{accn}_{isolation_country}_{collection_date}"

                        # Criar um objeto SeqRecord com ID e sequência
                        seq_record = SeqRecord(Seq(seq_sequence), id=novo_id, description='')
                        sequencias_novas.append(seq_record)
                    else:
                        (f"{accn} - {collection_date} - fora do formato AAAA/MM/DD.")
                else:
                    print(f"Nenhuma correspondência encontrada para o accn {accn}.")

            # Ordena a lista com base na função de chave
            sequencias_ordenadas = sorted(sequencias_novas, key=chave_ordenacao)
            # for sequencia in itens_ordenados:
            #     print(sequencia.id)

            # Sobrescrever o arquivo original com as sequências filtradas
            with open(path_sequencias_novas, "w") as novo_arquivo:
                SeqIO.write(sequencias_ordenadas, novo_arquivo, "fasta")


print('Concluído!')


Analisando sequências em B.1.1.529
Analisando sequências em B.1.1.7


/tmp/ipykernel_7352/540187629.py:24: DtypeWarning: Columns (35,44,64,80,82,84) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_planilha = pd.read_csv(arquivo_csv)


Analisando sequências em B.1.617.2


/tmp/ipykernel_7352/540187629.py:24: DtypeWarning: Columns (35,44,64,78,82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_planilha = pd.read_csv(arquivo_csv)


Analisando sequências em P.1


/tmp/ipykernel_7352/540187629.py:24: DtypeWarning: Columns (35,64) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_planilha = pd.read_csv(arquivo_csv)


Analisando sequências em B.1.351
Concluído!
